In [1]:
import pandas as pd
import nltk as tm
from nltk.corpus import stopwords
import wordcloud
import matplotlib
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
pwd

'c:\\Users\\Stenh\\OneDrive - Uppsala universitet\\Termin 4\\Examensarbete_analys'

<h1>Inläsning och kontroll av data</h1>

In [3]:

df = pd.read_csv("data.csv", encoding="utf-8", delimiter = ",")

In [4]:
df.head()

,Tidstämpel,Kön,Vilken årskurs går du i?,Vilket av följande beskriver bäst ditt program?,Hur många invånare har orten där du bor?,Jag känner till GDPR.,Jag litar på att hemsidor hanterar mina personuppgifter på ett ansvarsfullt sätt.,"Jag väljer ""Godkänn alla cookies"".","Jag väljer ""Tillåt alla"".","Jag väljer ""Acceptera alla cookies"".",...,Jag är omsorgsfull med vilka uppgifter jag lämnar till hemsidor.,Jag bryr mig om att företag samlar in information om mig som person.,Jag bryr mig om att företag samlar in information om mitt beteende online.,Jag känner oro över att information som samlas in via cookies kan användas på ett felaktigt sätt.,Jag känner mig obekväm med att webbplatser spårar mitt beteende med hjälp av cookies.,När jag besöker en hemsida tycker att det är viktigt att jag får välja vilken information om mig som lagras.,Jag tycker att hemsidor bör förklara tydligt vad varje cookie används till.,Jag tycker att det är fel att samla in personuppgifter utan att informera användaren.,Jag tycker att företag inskränker på människors privatliv när de spårar aktivitet via cookies.,Jag tycker att det är rimligt att betala med personlig data istället för pengar för att använda tjänster online.
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-04-07 14.25.02,Vill ej ange,Årskurs 3,Teknik,0-10 000,2,3.0,2,2,2,...,3,5,3,3,3,5,2,3,3,3
4,2025-04-09 12.57.25,Man,Årskurs 3,Naturvetenskap,50-100 000,5,5.0,7,7,3,...,2,6,6,5,5,6,7,7,6,2


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 42 columns):
 #   Column                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                              --------------  -----  
 0   Tidstämpel                                                                                                          52 non-null     object 
 1   Kön                                                                                                                 52 non-null     object 
 2   Vilken årskurs går du i?                                                                                            52 non-null     object 
 3   Vilket av följande beskriver bäst ditt program?                                                                     52 non-null     object 
 4   Hur många invånare har

In [6]:
df.select_dtypes(include=["number"]).corr()

,Jag litar på att hemsidor hanterar mina personuppgifter på ett ansvarsfullt sätt.
Jag litar på att hemsidor hanterar mina personuppgifter på ett ansvarsfullt sätt.,1.0


<h1>Demografi</h1>

<h1>Kategorisering av frågor</h1>

<h1>Medelvärde av kategorier</h1>

<h1>Relabilitetstest</h1>

<h1>Korrelationstest</h1>

<h1>Regressionstest</h1>

<h1>T-test/ANOVA</h1>